### Testing Setup ###

In [ ]:
msg = "heyyyy notebook in vscode!!"
print(msg)

heyyyy notebook in vscode!!


### Actual Code ###

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [7]:
# Adjust path to wherever you placed the files in your Drive
train_path = "train.csv"
test_path = "test.csv"

# Reading only the first 5000 rows
train_df = pd.read_csv(train_path, nrows=5000, header=0, index_col="id")
test_df = pd.read_csv(test_path, header=0, index_col="id")
#test_labels_df = pd.read_csv(test_labels_path, header=0, index_col="id")

print("Train shape:", train_df.shape)
print(train_df.head())

print("Test shape:", test_df.shape)
print(test_df.head())

#print("Test labels shape:", test_labels_df.shape)
#print(test_labels_df)

Train shape: (5000, 7)
                                                       comment_text  toxic  \
id                                                                           
0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

                  severe_toxic  obscene  threat  insult  identity_hate  
id                                                                      
0000997932d777bf             0        0       0       0              0  
000103f0d9cfb60f             0        0       0       0              0  
000113f07ec002fd             0        0       0       0              0  
0001b41b1c6bb37e             0        0       0       0          

In [8]:
#split training data into x_train and y_train

x_train = train_df.drop(['toxic','severe_toxic','obscene','threat', 'insult'], axis=1)
y_train = train_df.drop(['comment_text'], axis=1)

print("x_train:")
print(x_train)

print("y_train:")
print(y_train)

x_train:
                                                       comment_text  \
id                                                                    
0000997932d777bf  Explanation\nWhy the edits made under my usern...   
000103f0d9cfb60f  D'aww! He matches this background colour I'm s...   
000113f07ec002fd  Hey man, I'm really not trying to edit war. It...   
0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...   
0001d958c54c6e35  You, sir, are my hero. Any chance you remember...   
...                                                             ...   
0d39d2eaa40a6241  "\n\nHello Marcruhwedell, and Welcome to Wikip...   
0d3b57f3b2db3f03  ...that's why I did ....cheers,   (talk · cont...   
0d3bb1f12d90a6a2  No, it's not a delayed reaction\n\nI just happ...   
0d3be13abf1bec52  "\n\nA slight difference with you\nI have to d...   
0d3c720db297b5a3  "\n\nNonsense.  The truth is NO schools give s...   

                  identity_hate  
id                               

In [ ]:
# Train sklearn model
from sklearn.linear_model import RidgeClassifier # just doing this one as a test -- cauases errors blehhhhh

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_modified = vectorizer.fit_transform(x_train['comment_text'])

model = RidgeClassifier(tol=1e-2, solver="sparse_cg")
model.fit(X_train_modified , y_train['toxic'])  #training toy model with one label

RidgeClassifier(solver='sparse_cg', tol=0.01)